In [1]:
import random
import time
import math

def generate_prime(bits):
    def is_prime(n, k=10):
        if n < 2: return False
        if n == 2 or n == 3: return True
        if n % 2 == 0: return False
        r, d = 0, n - 1
        while d % 2 == 0:
            r += 1
            d //= 2
        for _ in range(k):
            a = random.randrange(2, n - 1)
            x = pow(a, d, n)
            if x == 1 or x == n - 1: continue
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == n - 1: break
            else: return False
        return True
    while True:
        p = random.getrandbits(bits) | (1 << bits - 1) | 1
        if is_prime(p): return p

def mod_inverse(e, phi):
    def extended_gcd(a, b):
        if a == 0: return b, 0, 1
        g, x1, y1 = extended_gcd(b % a, a)
        return g, y1 - (b // a) * x1, x1
    _, x, _ = extended_gcd(e % phi, phi)
    return (x % phi + phi) % phi


# Instrucciones como números
# Cada instrucción es un número que codifica: opcode + src1 + src2 + dst
OPCODES = ['add', 'sub', 'mul', 'mod', 'gcd', 'pow', 'sqrt', 'shr', 'xor']
REGS = ['a', 'b', 'c', 'n']

NUM_OPCODES = len(OPCODES)
NUM_REGS = len(REGS)

# Cada instrucción = opcode * 64 + src1 * 16 + src2 * 4 + dst
# Total posibilidades por instrucción = 9 * 4 * 4 * 4 = 576
INSTRUCTIONS_PER_SLOT = NUM_OPCODES * NUM_REGS * NUM_REGS * NUM_REGS

def decode_instruction(num):
    """Convierte número a instrucción"""
    num = num % INSTRUCTIONS_PER_SLOT
    dst = num % NUM_REGS
    num //= NUM_REGS
    src2 = num % NUM_REGS
    num //= NUM_REGS
    src1 = num % NUM_REGS
    num //= NUM_REGS
    opcode = num % NUM_OPCODES
    return OPCODES[opcode], REGS[src1], REGS[src2], REGS[dst]

def encode_instruction(opcode_idx, src1_idx, src2_idx, dst_idx):
    """Convierte instrucción a número"""
    return opcode_idx * (NUM_REGS ** 3) + src1_idx * (NUM_REGS ** 2) + src2_idx * NUM_REGS + dst_idx

def algorithm_from_number(num, num_instructions=10):
    """Convierte un número grande a una secuencia de instrucciones"""
    instructions = []
    for _ in range(num_instructions):
        instructions.append(decode_instruction(num % INSTRUCTIONS_PER_SLOT))
        num //= INSTRUCTIONS_PER_SLOT
    return instructions

def execute_algorithm(instructions, n, max_iter=50000):
    """Ejecuta secuencia de instrucciones"""
    regs = {
        'a': 2,
        'b': 2,
        'c': int(math.isqrt(n)) + 1,
        'n': n
    }

    for iteration in range(max_iter):
        for opcode, src1, src2, dst in instructions:
            try:
                v1 = regs[src1]
                v2 = regs[src2]

                if opcode == 'add':
                    result = (v1 + v2) % n
                elif opcode == 'sub':
                    result = (v1 - v2) % n
                elif opcode == 'mul':
                    result = (v1 * v2) % n
                elif opcode == 'mod':
                    result = v1 % v2 if v2 > 0 else v1
                elif opcode == 'gcd':
                    result = math.gcd(v1, n)
                elif opcode == 'pow':
                    result = pow(v1, 2, n)
                elif opcode == 'sqrt':
                    result = int(math.isqrt(v1)) if v1 > 0 else 1
                elif opcode == 'shr':
                    result = v1 >> 1
                elif opcode == 'xor':
                    result = v1 ^ v2
                else:
                    result = v1

                if result > 0:
                    regs[dst] = result

                # Check factor
                if 1 < result < n and n % result == 0:
                    return result, n // result, iteration

            except:
                continue

        # Check all registers
        for r in ['a', 'b', 'c']:
            val = regs[r]
            if 1 < val < n and n % val == 0:
                return val, n // val, iteration

    return None, None, max_iter

def test_algorithm_number(algo_num, test_cases, num_instructions=10):
    """Prueba un algoritmo codificado como número"""
    instructions = algorithm_from_number(algo_num, num_instructions)

    successes = 0
    for n, p, q in test_cases:
        pf, qf, _ = execute_algorithm(instructions, n, max_iter=1000)
        if pf and pf * qf == n:
            successes += 1

    return successes == len(test_cases), successes

def kaoru_binary_search(test_cases, num_instructions=10):
    """
    Búsqueda binaria sobre el espacio de algoritmos.
    El espacio es [0, INSTRUCTIONS_PER_SLOT ^ num_instructions)
    """

    space_size = INSTRUCTIONS_PER_SLOT ** num_instructions

    print(f"Espacio de búsqueda: {space_size} algoritmos posibles")
    print(f"Instrucciones por algoritmo: {num_instructions}")
    print(f"Buscando con búsqueda binaria...\n")

    low = 0
    high = space_size - 1

    best_algo = None
    best_score = 0
    iterations = 0

    while low <= high:
        iterations += 1
        mid = (low + high) // 2

        # Evaluar algoritmo en mid
        works, score = test_algorithm_number(mid, test_cases, num_instructions)

        if iterations % 1000 == 0:
            print(f"Iteración {iterations}: probando algoritmo #{mid}, score={score}/{len(test_cases)}")

        if works:
            print(f"\n{'=' * 50}")
            print(f"ALGORITMO ENCONTRADO!")
            print(f"{'=' * 50}")
            print(f"Número del algoritmo: {mid}")
            print(f"Iteraciones de búsqueda: {iterations}")

            instructions = algorithm_from_number(mid, num_instructions)
            print(f"\nInstrucciones:")
            for i, (op, s1, s2, d) in enumerate(instructions):
                print(f"  {i+1}. {d} = {op}({s1}, {s2})")

            return mid, instructions

        if score > best_score:
            best_score = score
            best_algo = mid

        # Decisión de búsqueda binaria basada en el score
        # Si score > 0, buscar en vecindad
        # Si score = 0, saltar más lejos

        if score > 0:
            # Hay algo prometedor, buscar cerca
            high = mid - 1
        else:
            # Nada, saltar
            low = mid + 1

    # Si búsqueda binaria pura no funciona, probar vecindad del mejor
    if best_algo:
        print(f"\nBúsqueda en vecindad del mejor ({best_score}/{len(test_cases)})...")

        for offset in range(-10000, 10001):
            candidate = best_algo + offset
            if candidate < 0 or candidate >= space_size:
                continue

            works, score = test_algorithm_number(candidate, test_cases, num_instructions)

            if works:
                print(f"\n{'=' * 50}")
                print(f"ALGORITMO ENCONTRADO!")
                print(f"{'=' * 50}")

                instructions = algorithm_from_number(candidate, num_instructions)
                print(f"\nInstrucciones:")
                for i, (op, s1, s2, d) in enumerate(instructions):
                    print(f"  {i+1}. {d} = {op}({s1}, {s2})")

                return candidate, instructions

    return None, None


def kaoru_reduction_rsa1024():
    print("=" * 60)
    print("KAORU DISCRETE REDUCTION")
    print("Búsqueda Binaria sobre Espacio de Algoritmos")
    print("RSA-1024")
    print("=" * 60)

    # Generar RSA-1024
    print("\nGenerando RSA-1024...")
    p = generate_prime(512)
    q = generate_prime(512)
    n = p * q
    e = 65537

    print(f"n = {str(n)[:50]}...")
    print(f"Bits: {n.bit_length()}")

    secret = 1337
    phi = (p - 1) * (q - 1)
    d = mod_inverse(e, phi)
    ciphertext = pow(secret, e, n)

    # Crear test cases pequeños
    print("\nCreando casos de prueba pequeños...")
    test_cases = []
    for bits in [8, 10, 12]:
        tp = generate_prime(bits)
        tq = generate_prime(bits)
        test_cases.append((tp * tq, tp, tq))
        print(f"  {tp * tq} = {tp} × {tq}")

    # Búsqueda binaria
    print("\n" + "=" * 60)
    print("FASE 1: Búsqueda de algoritmo")
    print("=" * 60)

    algo_num, instructions = kaoru_binary_search(test_cases, num_instructions=8)

    if instructions:
        # Aplicar a RSA-1024
        print("\n" + "=" * 60)
        print("FASE 2: Aplicando a RSA-1024")
        print("=" * 60)

        print("\nEjecutando...")
        start = time.time()
        p_found, q_found, iters = execute_algorithm(instructions, n, max_iter=10000000)
        elapsed = time.time() - start

        if p_found:
            print(f"\nFACTOR ENCONTRADO!")
            print(f"Tiempo: {elapsed:.2f}s")
            print(f"p = {p_found}")
            print(f"q = {q_found}")

            phi_r = (p_found - 1) * (q_found - 1)
            d_r = mod_inverse(e, phi_r)
            decrypted = pow(ciphertext, d_r, n)

            print(f"\nSecreto original: {secret}")
            print(f"Descifrado: {decrypted}")
            print(f"Éxito: {decrypted == secret}")
        else:
            print(f"\nEjecutó {iters} iteraciones en {elapsed:.2f}s")
            print("No encontró factor")
    else:
        print("\nNo se encontró algoritmo viable")


kaoru_reduction_rsa1024()

KAORU DISCRETE REDUCTION
Búsqueda Binaria sobre Espacio de Algoritmos
RSA-1024

Generando RSA-1024...
n = 10160325768167043379113190500186463894905245938688...
Bits: 1024

Creando casos de prueba pequeños...
  32231 = 167 × 193
  579811 = 1019 × 569
  7427107 = 2789 × 2663

FASE 1: Búsqueda de algoritmo
Espacio de búsqueda: 12116574790945106558976 algoritmos posibles
Instrucciones por algoritmo: 8
Buscando con búsqueda binaria...


No se encontró algoritmo viable


In [2]:
import random
import time
import math

def generate_prime(bits):
    def is_prime(n, k=10):
        if n < 2: return False
        if n == 2 or n == 3: return True
        if n % 2 == 0: return False
        r, d = 0, n - 1
        while d % 2 == 0:
            r += 1
            d //= 2
        for _ in range(k):
            a = random.randrange(2, n - 1)
            x = pow(a, d, n)
            if x == 1 or x == n - 1: continue
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == n - 1: break
            else: return False
        return True
    while True:
        p = random.getrandbits(bits) | (1 << bits - 1) | 1
        if is_prime(p): return p

def mod_inverse(e, phi):
    def extended_gcd(a, b):
        if a == 0: return b, 0, 1
        g, x1, y1 = extended_gcd(b % a, a)
        return g, y1 - (b // a) * x1, x1
    _, x, _ = extended_gcd(e % phi, phi)
    return (x % phi + phi) % phi


OPCODES = ['add', 'sub', 'mul', 'mod', 'gcd', 'pow', 'sqrt', 'shr', 'xor']
REGS = ['a', 'b', 'c', 'n']

NUM_OPCODES = len(OPCODES)
NUM_REGS = len(REGS)
INSTRUCTIONS_PER_SLOT = NUM_OPCODES * NUM_REGS * NUM_REGS * NUM_REGS

def decode_instruction(num):
    num = num % INSTRUCTIONS_PER_SLOT
    dst = num % NUM_REGS
    num //= NUM_REGS
    src2 = num % NUM_REGS
    num //= NUM_REGS
    src1 = num % NUM_REGS
    num //= NUM_REGS
    opcode = num % NUM_OPCODES
    return OPCODES[opcode], REGS[src1], REGS[src2], REGS[dst]

def algorithm_from_number(num, num_instructions=10):
    instructions = []
    for _ in range(num_instructions):
        instructions.append(decode_instruction(num % INSTRUCTIONS_PER_SLOT))
        num //= INSTRUCTIONS_PER_SLOT
    return instructions

def execute_algorithm(instructions, n, max_iter=50000):
    regs = {
        'a': 2,
        'b': 2,
        'c': int(math.isqrt(n)) + 1,
        'n': n
    }

    for iteration in range(max_iter):
        for opcode, src1, src2, dst in instructions:
            try:
                v1 = regs[src1]
                v2 = regs[src2]

                if opcode == 'add':
                    result = (v1 + v2) % n
                elif opcode == 'sub':
                    result = (v1 - v2) % n
                elif opcode == 'mul':
                    result = (v1 * v2) % n
                elif opcode == 'mod':
                    result = v1 % v2 if v2 > 0 else v1
                elif opcode == 'gcd':
                    result = math.gcd(v1, n)
                elif opcode == 'pow':
                    result = pow(v1, 2, n)
                elif opcode == 'sqrt':
                    result = int(math.isqrt(v1)) if v1 > 0 else 1
                elif opcode == 'shr':
                    result = v1 >> 1
                elif opcode == 'xor':
                    result = v1 ^ v2
                else:
                    result = v1

                if result > 0:
                    regs[dst] = result

                if 1 < result < n and n % result == 0:
                    return result, n // result, iteration

            except:
                continue

        for r in ['a', 'b', 'c']:
            val = regs[r]
            if 1 < val < n and n % val == 0:
                return val, n // val, iteration

    return None, None, max_iter

def test_algorithm_number(algo_num, test_cases, num_instructions=10):
    instructions = algorithm_from_number(algo_num, num_instructions)

    successes = 0
    for n, p, q in test_cases:
        pf, qf, _ = execute_algorithm(instructions, n, max_iter=1000)
        if pf and pf * qf == n:
            successes += 1

    return successes == len(test_cases), successes, instructions

def kaoru_binary_subdivision(test_cases, num_instructions=6):
    """
    Búsqueda binaria por subdivisión recursiva del espacio.
    Divide el espacio en mitades, muestrea cada mitad,
    se enfoca en la mejor mitad.
    """

    space_size = INSTRUCTIONS_PER_SLOT ** num_instructions

    print(f"Espacio: {space_size} algoritmos")
    print(f"Instrucciones: {num_instructions}")
    print()

    tested = 0

    def recursive_search(low, high, depth=0):
        nonlocal tested

        if high - low < 1:
            return None, None

        indent = "  " * depth
        range_size = high - low + 1

        print(f"{indent}Nivel {depth}: rango [{low}, {high}] (tamaño: {range_size})")

        # Muestrear en este rango
        samples_per_range = min(100, range_size)

        best_score = 0
        best_algo = None
        best_instructions = None

        for _ in range(samples_per_range):
            sample = random.randint(low, high)
            tested += 1

            works, score, instructions = test_algorithm_number(sample, test_cases, num_instructions)

            if works:
                print(f"\n{indent}¡ENCONTRADO en muestra #{tested}!")
                print(f"{indent}Algoritmo: {sample}")
                return sample, instructions

            if score > best_score:
                best_score = score
                best_algo = sample
                best_instructions = instructions

        print(f"{indent}Mejor score en rango: {best_score}/{len(test_cases)}")

        if best_score == 0 and depth < 10:
            # Dividir en mitades y buscar en ambas
            mid = (low + high) // 2

            # Buscar en mitad izquierda
            result = recursive_search(low, mid, depth + 1)
            if result[0]:
                return result

            # Buscar en mitad derecha
            result = recursive_search(mid + 1, high, depth + 1)
            if result[0]:
                return result

        elif best_score > 0 and depth < 15:
            # Enfocarse alrededor del mejor
            window = max(1000, range_size // 100)
            new_low = max(low, best_algo - window)
            new_high = min(high, best_algo + window)

            print(f"{indent}Enfocando alrededor de {best_algo}")

            result = recursive_search(new_low, new_high, depth + 1)
            if result[0]:
                return result

        return None, None

    print("Iniciando búsqueda binaria recursiva...\n")
    return recursive_search(0, space_size - 1)

def kaoru_reduction_rsa1024():
    print("=" * 60)
    print("KAORU DISCRETE REDUCTION")
    print("Búsqueda Binaria por Subdivisión")
    print("RSA-1024")
    print("=" * 60)

    print("\nGenerando RSA-1024...")
    p = generate_prime(512)
    q = generate_prime(512)
    n = p * q
    e = 65537

    print(f"n = {str(n)[:50]}...")
    print(f"Bits: {n.bit_length()}")

    secret = 1337
    phi = (p - 1) * (q - 1)
    d = mod_inverse(e, phi)
    ciphertext = pow(secret, e, n)

    print("\nCasos de prueba:")
    test_cases = []
    for bits in [8, 10, 12]:
        tp = generate_prime(bits)
        tq = generate_prime(bits)
        tn = tp * tq
        test_cases.append((tn, tp, tq))
        print(f"  {tn} = {tp} × {tq}")

    print("\n" + "=" * 60)
    print("FASE 1: Búsqueda de algoritmo")
    print("=" * 60 + "\n")

    start_search = time.time()
    algo_num, instructions = kaoru_binary_subdivision(test_cases, num_instructions=6)
    search_time = time.time() - start_search

    if instructions:
        print("\n" + "=" * 60)
        print("ALGORITMO ENCONTRADO")
        print("=" * 60)
        print(f"Tiempo de búsqueda: {search_time:.2f}s")
        print(f"Número: {algo_num}")
        print("\nInstrucciones:")
        for i, (op, s1, s2, d) in enumerate(instructions):
            print(f"  {i+1}. {d} = {op}({s1}, {s2})")

        print("\n" + "=" * 60)
        print("FASE 2: Aplicando a RSA-1024")
        print("=" * 60)

        print("\nEjecutando...")
        start = time.time()
        p_found, q_found, iters = execute_algorithm(instructions, n, max_iter=10000000)
        elapsed = time.time() - start

        if p_found:
            print(f"\n¡FACTOR ENCONTRADO!")
            print(f"Tiempo: {elapsed:.2f}s")
            print(f"Iteraciones: {iters}")
            print(f"p = {str(p_found)[:50]}...")
            print(f"q = {str(q_found)[:50]}...")
            print(f"Verificación: {p_found * q_found == n}")

            phi_r = (p_found - 1) * (q_found - 1)
            d_r = mod_inverse(e, phi_r)
            decrypted = pow(ciphertext, d_r, n)

            print(f"\nSecreto: {secret}")
            print(f"Descifrado: {decrypted}")
            print(f"¡ÉXITO!: {decrypted == secret}")
        else:
            print(f"\nIteraciones: {iters}")
            print(f"Tiempo: {elapsed:.2f}s")
            print("El algoritmo funciona en casos pequeños")
            print("pero necesita más iteraciones para RSA-1024")
    else:
        print(f"\nNo encontrado en {search_time:.2f}s")
        print("Aumentar profundidad o muestras por nivel")

kaoru_reduction_rsa1024()

KAORU DISCRETE REDUCTION
Búsqueda Binaria por Subdivisión
RSA-1024

Generando RSA-1024...
n = 67213906049280358756656315859161882269746685891759...
Bits: 1023

Casos de prueba:
  20413 = 149 × 137
  401227 = 607 × 661
  7902971 = 2939 × 2689

FASE 1: Búsqueda de algoritmo

Espacio: 36520347436056576 algoritmos
Instrucciones: 6

Iniciando búsqueda binaria recursiva...

Nivel 0: rango [0, 36520347436056575] (tamaño: 36520347436056576)

¡ENCONTRADO en muestra #9!
Algoritmo: 7296527037666369

ALGORITMO ENCONTRADO
Tiempo de búsqueda: 0.04s
Número: 7296527037666369

Instrucciones:
  1. b = mul(a, a)
  2. n = mod(b, b)
  3. n = sub(n, c)
  4. n = sqrt(a, b)
  5. c = add(c, n)
  6. n = sub(n, a)

FASE 2: Aplicando a RSA-1024

Ejecutando...

Iteraciones: 10000000
Tiempo: 71.87s
El algoritmo funciona en casos pequeños
pero necesita más iteraciones para RSA-1024


busqueda binaria para ENCONTRAR un algoritmo mas eficiente

In [ ]:
import random
import time
import math

def generate_prime(bits):
    def is_prime(n, k=10):
        if n < 2: return False
        if n == 2 or n == 3: return True
        if n % 2 == 0: return False
        r, d = 0, n - 1
        while d % 2 == 0:
            r += 1
            d //= 2
        for _ in range(k):
            a = random.randrange(2, n - 1)
            x = pow(a, d, n)
            if x == 1 or x == n - 1: continue
            for _ in range(r - 1):
                x = pow(x, 2, n)
                if x == n - 1: break
            else: return False
        return True
    while True:
        p = random.getrandbits(bits) | (1 << bits - 1) | 1
        if is_prime(p): return p

def mod_inverse(e, phi):
    def extended_gcd(a, b):
        if a == 0: return b, 0, 1
        g, x1, y1 = extended_gcd(b % a, a)
        return g, y1 - (b // a) * x1, x1
    _, x, _ = extended_gcd(e % phi, phi)
    return (x % phi + phi) % phi


OPCODES = ['add', 'sub', 'mul', 'mod', 'gcd', 'pow', 'sqrt', 'shr', 'xor']
REGS = ['a', 'b', 'c', 'n']

NUM_OPCODES = len(OPCODES)
NUM_REGS = len(REGS)
INSTRUCTIONS_PER_SLOT = NUM_OPCODES * NUM_REGS * NUM_REGS * NUM_REGS

def decode_instruction(num):
    num = num % INSTRUCTIONS_PER_SLOT
    dst = num % NUM_REGS
    num //= NUM_REGS
    src2 = num % NUM_REGS
    num //= NUM_REGS
    src1 = num % NUM_REGS
    num //= NUM_REGS
    opcode = num % NUM_OPCODES
    return OPCODES[opcode], REGS[src1], REGS[src2], REGS[dst]

def algorithm_from_number(num, num_instructions=10):
    instructions = []
    for _ in range(num_instructions):
        instructions.append(decode_instruction(num % INSTRUCTIONS_PER_SLOT))
        num //= INSTRUCTIONS_PER_SLOT
    return instructions

def execute_algorithm(instructions, n, max_iter=50000, show_progress=False):
    regs = {
        'a': 2,
        'b': 2,
        'c': int(math.isqrt(n)) + 1,
        'n': n
    }

    last_print = time.time()

    for iteration in range(max_iter):
        if show_progress and iteration % 1000000 == 0 and iteration > 0:
            if time.time() - last_print > 2:
                print(f"  Iteración {iteration:,}... (a={regs['a'] % 10000}, b={regs['b'] % 10000}, c={regs['c'] % 10000})")
                last_print = time.time()

        for opcode, src1, src2, dst in instructions:
            try:
                v1 = regs[src1]
                v2 = regs[src2]

                if opcode == 'add':
                    result = (v1 + v2) % n
                elif opcode == 'sub':
                    result = (v1 - v2) % n
                elif opcode == 'mul':
                    result = (v1 * v2) % n
                elif opcode == 'mod':
                    result = v1 % v2 if v2 > 0 else v1
                elif opcode == 'gcd':
                    result = math.gcd(v1, n)
                elif opcode == 'pow':
                    result = pow(v1, 2, n)
                elif opcode == 'sqrt':
                    result = int(math.isqrt(abs(v1))) if v1 != 0 else 1
                elif opcode == 'shr':
                    result = v1 >> 1
                elif opcode == 'xor':
                    result = v1 ^ v2
                else:
                    result = v1

                if result > 0:
                    regs[dst] = result

                if 1 < result < n and n % result == 0:
                    return result, n // result, iteration

            except:
                continue

        for r in ['a', 'b', 'c']:
            val = regs[r]
            if 1 < val < n and n % val == 0:
                return val, n // val, iteration

    return None, None, max_iter

def test_algorithm_number(algo_num, test_cases, num_instructions=10):
    instructions = algorithm_from_number(algo_num, num_instructions)

    successes = 0
    for n, p, q in test_cases:
        pf, qf, _ = execute_algorithm(instructions, n, max_iter=5000)
        if pf and pf * qf == n:
            successes += 1

    return successes == len(test_cases), successes, instructions

def kaoru_binary_subdivision(test_cases, num_instructions=6):
    space_size = INSTRUCTIONS_PER_SLOT ** num_instructions

    print(f"Espacio: {space_size} algoritmos")
    print(f"Instrucciones: {num_instructions}")
    print()

    tested = 0

    def recursive_search(low, high, depth=0):
        nonlocal tested

        if high - low < 1:
            return None, None

        indent = "  " * depth
        range_size = high - low + 1

        print(f"{indent}Nivel {depth}: rango [{low}, {high}] (tamaño: {range_size})")

        samples_per_range = min(200, range_size)

        best_score = 0
        best_algo = None
        best_instructions = None

        for _ in range(samples_per_range):
            sample = random.randint(low, high)
            tested += 1

            works, score, instructions = test_algorithm_number(sample, test_cases, num_instructions)

            if works:
                print(f"\n{indent}¡ENCONTRADO en muestra #{tested}!")
                print(f"{indent}Algoritmo: {sample}")
                return sample, instructions

            if score > best_score:
                best_score = score
                best_algo = sample
                best_instructions = instructions

        print(f"{indent}Mejor score: {best_score}/{len(test_cases)}")

        if best_score == 0 and depth < 8:
            mid = (low + high) // 2

            result = recursive_search(low, mid, depth + 1)
            if result[0]:
                return result

            result = recursive_search(mid + 1, high, depth + 1)
            if result[0]:
                return result

        elif best_score > 0 and depth < 12:
            window = max(10000, range_size // 50)
            new_low = max(low, best_algo - window)
            new_high = min(high, best_algo + window)

            print(f"{indent}Enfocando alrededor de {best_algo}")

            result = recursive_search(new_low, new_high, depth + 1)
            if result[0]:
                return result

        return None, None

    print("Iniciando búsqueda...\n")
    return recursive_search(0, space_size - 1)

def kaoru_reduction_rsa1024():
    print("=" * 60)
    print("KAORU DISCRETE REDUCTION")
    print("RSA-1024 - Búsqueda Binaria de Algoritmos")
    print("=" * 60)

    print("\nGenerando RSA-1024...")
    p = generate_prime(512)
    q = generate_prime(512)
    n = p * q
    e = 65537

    print(f"n = {str(n)[:50]}...")
    print(f"Bits: {n.bit_length()}")

    secret = 42424242
    phi = (p - 1) * (q - 1)
    d = mod_inverse(e, phi)
    ciphertext = pow(secret, e, n)

    # Casos de prueba más grandes esta vez
    print("\nCasos de prueba (más grandes):")
    test_cases = []
    for bits in [12, 16, 20]:
        tp = generate_prime(bits)
        tq = generate_prime(bits)
        tn = tp * tq
        test_cases.append((tn, tp, tq))
        print(f"  {tn} ({tn.bit_length()} bits) = {tp} × {tq}")

    print("\n" + "=" * 60)
    print("FASE 1: Búsqueda de algoritmo")
    print("=" * 60 + "\n")

    start_search = time.time()
    algo_num, instructions = kaoru_binary_subdivision(test_cases, num_instructions=8)
    search_time = time.time() - start_search

    if instructions:
        print("\n" + "=" * 60)
        print("ALGORITMO ENCONTRADO")
        print("=" * 60)
        print(f"Tiempo de búsqueda: {search_time:.2f}s")
        print(f"Número del algoritmo: {algo_num}")
        print("\nInstrucciones:")
        for i, (op, s1, s2, d) in enumerate(instructions):
            print(f"  {i+1}. {d} = {op}({s1}, {s2})")

        # Verificar en casos de prueba
        print("\nVerificando funcionamiento:")
        for tn, tp, tq in test_cases:
            pf, qf, iters = execute_algorithm(instructions, tn, max_iter=10000)
            status = "✓" if pf and pf * qf == tn else "✗"
            print(f"  {status} n={tn} en {iters} iteraciones")

        print("\n" + "=" * 60)
        print("FASE 2: Aplicando a RSA-1024")
        print("=" * 60)

        MAX_ITERATIONS = 100000000  # 100 millones
        print(f"\nEjecutando (máx {MAX_ITERATIONS:,} iteraciones)...")
        print("Esto puede tomar varios minutos...\n")

        start = time.time()
        p_found, q_found, iters = execute_algorithm(
            instructions, n,
            max_iter=MAX_ITERATIONS,
            show_progress=True
        )
        elapsed = time.time() - start

        print(f"\n{'=' * 60}")
        if p_found:
            print("¡¡¡ÉXITO TOTAL!!!")
            print("=" * 60)
            print(f"Tiempo: {elapsed:.2f}s")
            print(f"Iteraciones: {iters:,}")
            print(f"\np = {str(p_found)[:60]}...")
            print(f"q = {str(q_found)[:60]}...")
            print(f"\nVerificación: p × q = n: {p_found * q_found == n}")

            phi_r = (p_found - 1) * (q_found - 1)
            d_r = mod_inverse(e, phi_r)
            decrypted = pow(ciphertext, d_r, n)

            print(f"\nDescifrando mensaje:")
            print(f"  Secreto original: {secret}")
            print(f"  Descifrado: {decrypted}")
            print(f"  ¡ÉXITO!: {decrypted == secret}")

            print("\n" + "=" * 60)
            print("P = NP DEMOSTRADO")
            print("=" * 60)
        else:
            print("LÍMITE ALCANZADO")
            print("=" * 60)
            print(f"Iteraciones: {iters:,}")
            print(f"Tiempo: {elapsed:.2f}s")
            print(f"Velocidad: {iters/elapsed:,.0f} iter/s")
            print("\nEl algoritmo funciona para números pequeños.")
            print(f"Para RSA-1024 se necesitarían ~{(2**512)//(iters/elapsed):,.0e} años")
    else:
        print(f"\nNo encontrado en {search_time:.2f}s")

kaoru_reduction_rsa1024()

KAORU DISCRETE REDUCTION
RSA-1024 - Búsqueda Binaria de Algoritmos

Generando RSA-1024...
n = 56488116166716261159844594949790678772166459088781...
Bits: 1023

Casos de prueba (más grandes):
  7868797 (23 bits) = 3347 × 2351
  2518758079 (32 bits) = 56501 × 44579
  390624670741 (39 bits) = 727121 × 537221

FASE 1: Búsqueda de algoritmo

Espacio: 12116574790945106558976 algoritmos
Instrucciones: 8

Iniciando búsqueda...

Nivel 0: rango [0, 12116574790945106558975] (tamaño: 12116574790945106558976)
Mejor score: 2/3
Enfocando alrededor de 494779763767055176119
  Nivel 1: rango [252448267948153044940, 737111259585957307298] (tamaño: 484662991637804262359)

  ¡ENCONTRADO en muestra #292!
  Algoritmo: 565692646750867676844

ALGORITMO ENCONTRADO
Tiempo de búsqueda: 18.32s
Número del algoritmo: 565692646750867676844

Instrucciones:
  1. a = sub(c, n)
  2. b = gcd(n, a)
  3. n = xor(c, a)
  4. n = pow(a, a)
  5. c = sub(b, c)
  6. b = shr(a, c)
  7. b = xor(a, a)
  8. c = add(b, c)

Verificando